## Exploring presolving and plugins options

With presolving, we indicate all procedures that can be applied before the branch and bound starts.
Plugins concerns the application of constraints and variables changes during the execution of the B&B algorithm.

In [1]:
import sys
sys.path.append('..')
from pyscipopt import Model, quicksum
from pathlib import Path
import numpy as np
from typing import Union
from albp.data.utils import transitive_closure

In [2]:
class ALBP:

    def __init__(self, N: int = None, t: Union[np.ndarray, list] = None,
                 c: int = None, P: np.ndarray = None) -> None:
        self.N = N
        self.c = c
        self.t = t
        self.P = P
        self.OS = None

        self.name = None
        self.model = None

    def readFromFile(self, path: Union[str, Path]):
        if type(path) is str:
            path = Path(path)
        if path.suffix != ".alb":
            raise ValueError("The file format is not .alb.")
        self.name = path.stem
        with open(path, "r") as f:
            row = f.readline()
            while row:
                if "<number of tasks>" in row:
                    self.N = int(f.readline().strip())
                elif "<cycle time>" in row:
                    self.c = int(f.readline().strip())
                elif "<number of stations>" in row:
                    raise NotImplementedError
                elif "<order strength>" in row:
                    self.OS = float(f.readline().strip())
                elif "<task times>" in row:
                    self.t = np.array([int(f.readline().strip().split(" ")[-1])
                                       for _ in range(self.N)], dtype=int)
                elif "<precedence relations>" in row:
                    self.P = np.zeros((self.N, self.N), dtype=bool)
                    for p in f.readlines():
                        if p.strip():
                            p = tuple(p.strip().split(","))
                            self.P[int(p[0]) - 1, int(p[-1]) - 1] = 1
                        else: break
                elif "<sequence dependent time increments>" in row:
                    raise NotImplementedError
                elif "<linked tasks>" in row:
                    self.LT = np.zeros((self.N, self.N), dtype=bool)
                    for p in f.readlines():
                        if p.strip():
                            p = tuple(p.strip().split(","))
                            self.LT[int(p[-1]) - 1, int(p[0]) - 1] = 1
                        else: break
                elif "<total station cost>" in row:
                    raise NotImplementedError
                elif "<station cost per unit>" in row:
                    raise NotImplementedError
                elif "<total task cost>" in row:
                    raise NotImplementedError
                elif " <task cost per unit>" in row:
                    raise NotImplementedError
                # Parallel stations are duplicates of some serial station such
                # that the local cycle time is a multiple of the global cycle
                # time.
                # The maximal number of times a station can be installed in
                # parallel
                elif "<maximum degree of parallelism>" in row:
                    self.max_parallelism = int(f.readline().strip())
                elif "<number of equipments>" in row:
                    self.n_equipments = int(f.readline().strip())
                elif "<equipments per task>" in row:
                    self.equipment = \
                        np.array([int(f.readline().strip().split(" ")[-1])
                                  for _ in range(self.N)])
                elif "<number of task attributes>" in row:
                    raise NotImplementedError
                elif "<task attribute values>" in row:
                    raise NotImplementedError
                elif "<attribute bounds per station>" in row:
                    raise NotImplementedError
                elif "<incompatible tasks>" in row:
                    raise NotImplementedError
                elif "<end>" in row:
                    break
                row = f.readline()

    def writeModel(self):
        self.model = Model(problemName=self.name)
        model = self.model
        N = self.N
        t = self.t
        c = self.c
        P = self.P

        # Create the mip solver with the SCIP backend.
        if not model:
            return
        #TODO: think about the implementation of bounds, the assumption M = N is not efficient
        M = self.N

        #################
        # PREPROCESSING #
        #################
        # transitive closures predecessors
        Px = transitive_closure(P)
        # transitiive closures successors
        Fx = transitive_closure(P.T)
        # compute earliest and latest stations
        tau = self.t / c  # relative task time
        # compute earliest
        E = np.zeros(N, dtype=int)
        for i in range(N):
            E[i] = np.ceil(tau[i] + np.sum(tau[Px[i]]))
        # compute latest
        L = np.zeros(N, dtype=int)
        for i in range(N):
            L[i] = M + 1 - np.ceil(tau[i] + np.sum(tau[Fx[i]]))
        # compute feasible stations
        FS = np.zeros((N, N), dtype=bool)
        for i in range(N):
            FS[i, E[i]:L[i] + 1] = 1

        #############
        # VARIABLES #
        #############
        x = {}
        for i in range(N):
            for k in np.arange(N)[FS[i]]:
                x[i, k] = model.addVar(vtype="B", name="x_%s_%s" % (i, k))

        y = {}
        for k in range(M):
            y[k] = model.addVar(vtype="B", name="y_%s" % k)

        ###############
        # CONSRTAINTS #
        ###############
        # a product must be assigned to a machine
        for i in range(N):
            model.addCons(
                quicksum([x[i, k] for k in np.arange(N)[FS[i]]]) == 1
            )

        # cycle time must be respected
        for k in range(M):
            model.addCons(
                quicksum(
                    [t[i] * x[i, k] for i in np.arange(N)[FS[i]] if FS[i, k]]
                ) <= c * y[k]
            )

        # precedence constraints
        for i in range(N):
            for j in np.arange(N)[P[i]]:
                model.addCons(
                    quicksum([k * x[j, k] for k in np.arange(N)[FS[j]]]) <=
                    quicksum([k * x[i, k] for k in np.arange(N)[FS[i]]])
                )

        # write objective function
        model.setObjective(quicksum([y[k] for k in range(M)]), "minimize")

        # collect problem data to a dictionary and append them to the model
        model.data = {
            'N': self.N,
            'C': self.c,
            't': self.t,
            'P': self.P,
            'Px': Px,
            'Fx': Fx,
            'tau': tau,
            'E': E,
            'L': L,
            'FS': FS
        }

        return model

In [3]:
albp = ALBP()
albp.readFromFile("/home/miki/albp_solver/data/raw/albp-datasets/SALBP-1993/BOWMAN8.alb")
albp.writeModel()

print(albp.name)

BOWMAN8


In [6]:
print(albp.model.data)

{'N': 8, 'C': 1000, 't': array([11, 17,  9,  5,  8, 12, 10,  3]), 'P': array([[False, False, False, False, False, False, False, False],
       [False, False,  True,  True, False, False, False, False],
       [False, False, False, False,  True,  True, False, False],
       [False, False, False, False, False,  True, False, False],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False, False]]), 'Px': array([[False, False, False, False, False, False, False, False],
       [False, False,  True,  True,  True,  True,  True,  True],
       [False, False, False, False,  True,  True,  True,  True],
       [False, False, False, False, False,  True, False,  True],
       [False, False, False, False, False, False,  True, False],
       [False, False, False, False, False, False, False,  True],
       [Fal